In [88]:
import os
import glob
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import random
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, when, lower, trim, regexp_replace, split, udf, explode, array_contains
from pyspark.sql.types import FloatType, IntegerType, DateType
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler, StandardScaler
from pyspark.ml import Pipeline

In [9]:
# Initialize SparkSession
spark = SparkSession.builder \
    .appName("features_financials_preprocessing") \
    .master("local[*]") \
    .getOrCreate()

spark.sparkContext.setLogLevel("ERROR")

## Build Bronze Table

In [176]:
# Define input and output directories
input_file = "data/features_financials.csv"
output_directory = "datamart/bronze/features_financials/"
if not os.path.exists(output_directory):
    os.makedirs(output_directory)

In [177]:
# Load data
df = spark.read.option("header", "true").csv(input_file)

In [178]:
# Handle anomalies (e.g., negative values where not allowed)
df = df.withColumn("Annual_Income", when(col("Annual_Income") < 0, None).otherwise(col("Annual_Income"))) \
       .withColumn("Monthly_Inhand_Salary", when(col("Monthly_Inhand_Salary") < 0, None).otherwise(col("Monthly_Inhand_Salary"))) \
       .withColumn("Num_Bank_Accounts", when(col("Num_Bank_Accounts") < 0, None).otherwise(col("Num_Bank_Accounts"))) \
       .withColumn("Num_Credit_Card", when(col("Num_Credit_Card") < 0, None).otherwise(col("Num_Credit_Card"))) \
       .withColumn("Interest_Rate", when(col("Interest_Rate") < 0, None).otherwise(col("Interest_Rate"))) \
       .withColumn("Num_of_Loan", when(col("Num_of_Loan") < 0, None).otherwise(col("Num_of_Loan")))
df = df.na.drop()

In [179]:
# Replace "_" with null and handle invalid numeric values
for column in df.columns:
    df = df.withColumn(column, when(col(column) == "_", None).otherwise(col(column)))
    df = df.withColumn(column, regexp_replace(col(column), "_$", "").cast("string"))

In [180]:
# Drop rows with null values in critical columns
critical_columns = ["Customer_ID", "snapshot_date", "Annual_Income", "Monthly_Inhand_Salary"]
df = df.na.drop(subset=critical_columns)

In [181]:
# Correct data types
df = df.withColumn("snapshot_date", col("snapshot_date").cast(DateType())) \
       .withColumn("Annual_Income", col("Annual_Income").cast(FloatType())) \
       .withColumn("Monthly_Inhand_Salary", col("Monthly_Inhand_Salary").cast(FloatType())) \
       .withColumn("Num_Bank_Accounts", col("Num_Bank_Accounts").cast(IntegerType())) \
       .withColumn("Num_Credit_Card", col("Num_Credit_Card").cast(IntegerType())) \
       .withColumn("Interest_Rate", col("Interest_Rate").cast(FloatType())) \
       .withColumn("Num_of_Loan", col("Num_of_Loan").cast(IntegerType())) \
       .withColumn("Delay_from_due_date", col("Delay_from_due_date").cast(IntegerType())) \
       .withColumn("Num_of_Delayed_Payment", col("Num_of_Delayed_Payment").cast(IntegerType())) \
       .withColumn("Changed_Credit_Limit", col("Changed_Credit_Limit").cast(FloatType())) \
       .withColumn("Num_Credit_Inquiries", col("Num_Credit_Inquiries").cast(FloatType())) \
       .withColumn("Outstanding_Debt", col("Outstanding_Debt").cast(FloatType())) \
       .withColumn("Credit_Utilization_Ratio", col("Credit_Utilization_Ratio").cast(FloatType())) \
       .withColumn("Total_EMI_per_month", col("Total_EMI_per_month").cast(FloatType())) \
       .withColumn("Amount_invested_monthly", col("Amount_invested_monthly").cast(FloatType())) \
       .withColumn("Monthly_Balance", col("Monthly_Balance").cast(FloatType()))

## Build Silver Table

In [182]:
# Standardize text fields
text_columns = ["Type_of_Loan", "Credit_Mix", "Payment_of_Min_Amount", "Payment_Behaviour"]
for col_name in text_columns:
    df = df.withColumn(col_name, lower(trim(col(col_name))))

In [183]:
# Remove rows with abnormal value '!@9#%8' in Payment_Behaviour
df = df.filter(col("Payment_Behaviour") != '!@9#%8')

In [184]:
# Extract years and months from Credit_History_Age
def extract_years_months(age_str):
    if age_str is None:
        return None, None
    parts = age_str.split(" ")
    years = int(parts[0]) if parts[0].isdigit() else 0
    months = int(parts[3]) if len(parts) > 3 and parts[3].isdigit() else 0
    return years, months

extract_udf = udf(extract_years_months, "struct<years:int,months:int>")
df = df.withColumn("credit_history", extract_udf(col("Credit_History_Age"))) \
       .withColumn("Credit_History_Years", col("credit_history.years").cast(IntegerType())) \
       .withColumn("Credit_History_Months", col("credit_history.months").cast(IntegerType())) \
       .drop("credit_history")

In [185]:
# Calculate EMI to Income Ratio
df = df.withColumn("EMI_to_Income_Ratio", col("Total_EMI_per_month") / col("Monthly_Inhand_Salary"))

In [186]:
df.toPandas()['Payment_Behaviour'].unique()

array(['high_spent_medium_value_payments',
       'low_spent_medium_value_payments',
       'low_spent_small_value_payments',
       'high_spent_small_value_payments',
       'high_spent_large_value_payments',
       'low_spent_large_value_payments'], dtype=object)

In [187]:
df.toPandas()

,Customer_ID,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,Num_of_Loan,Type_of_Loan,Delay_from_due_date,Num_of_Delayed_Payment,...,Credit_History_Age,Payment_of_Min_Amount,Total_EMI_per_month,Amount_invested_monthly,Payment_Behaviour,Monthly_Balance,snapshot_date,Credit_History_Years,Credit_History_Months,EMI_to_Income_Ratio
0,CUS_0x1000,30625.939453,2706.161621,6,5,27.0,2,"credit-builder loan, and home equity loan",57,26,...,10 Years and 9 Months,yes,42.941090,77.314278,high_spent_medium_value_payments,400.360809,2023-05-01,10,9,0.015868
1,CUS_0x1009,52312.679688,4250.390137,6,5,17.0,4,"not specified, home equity loan, credit-builde...",5,18,...,31 Years and 0 Months,yes,108.366470,58.660191,high_spent_medium_value_payments,508.012329,2025-01-01,31,0,0.025496
2,CUS_0x1011,58918.468750,5208.872559,3,3,17.0,3,"student loan, credit-builder loan, and debt co...",27,13,...,15 Years and 10 Months,yes,123.434937,383.350830,low_spent_medium_value_payments,294.101471,2023-11-01,15,10,0.023697
3,CUS_0x1013,98620.976562,7962.415039,3,3,6.0,3,"student loan, debt consolidation loan, and per...",12,9,...,17 Years and 10 Months,no,228.018082,332.333710,high_spent_medium_value_payments,485.889709,2023-12-01,17,10,0.028637
4,CUS_0x1018,61194.808594,5014.567383,7,7,23.0,8,"credit-builder loan, payday loan, home equity ...",19,22,...,14 Years and 3 Months,yes,225.368698,166.530624,high_spent_medium_value_payments,359.557434,2023-11-01,14,3,0.044943
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9768,CUS_0xff3,17032.785156,1176.398804,0,6,2.0,3,"personal loan, mortgage loan, and auto loan",13,7,...,17 Years and 3 Months,no,33.299763,81.195854,low_spent_small_value_payments,293.144257,2024-06-01,17,3,0.028307
9769,CUS_0xff4,25546.259766,2415.854980,8,7,14.0,5,"not specified, student loan, student loan, cre...",15,13,...,18 Years and 9 Months,yes,101.328636,189.815857,low_spent_medium_value_payments,230.440994,2024-12-01,18,9,0.041943
9770,CUS_0xff6,117639.921875,9727.327148,5,6,1.0,2,"home equity loan, and auto loan",-3,7,...,24 Years and 11 Months,no,126.638451,534.088501,low_spent_medium_value_payments,592.005676,2024-10-01,24,11,0.013019
9771,CUS_0xffc,60877.171875,5218.097656,6,8,27.0,8,"credit-builder loan, payday loan, not specifie...",46,14,...,13 Years and 1 Months,yes,272.809174,46.425613,high_spent_large_value_payments,442.574982,2024-01-01,13,1,0.052281


## Build Gold Table

In [188]:
# Split the "Type_of_Loan" column into an array of loan types
df = df.withColumn("Loan_Types", split(col("Type_of_Loan"), ", and |, "))

# Get unique loan types
unique_loan_types = df.select(explode(col("Loan_Types")).alias("Loan_Type")).distinct().collect()
unique_loan_types = [row["Loan_Type"] for row in unique_loan_types]

# Create binary columns for each unique loan type
for loan_type in unique_loan_types:
    df = df.withColumn(loan_type, array_contains(col("Loan_Types"), loan_type).cast("int"))

# Optionally, drop the temporary "Loan_Types" column
df = df.drop("Loan_Types", "Type_of_Loan")
df.toPandas()

,Customer_ID,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,Num_of_Loan,Delay_from_due_date,Num_of_Delayed_Payment,Changed_Credit_Limit,...,EMI_to_Income_Ratio,debt consolidation loan,personal loan,payday loan,mortgage loan,not specified,credit-builder loan,auto loan,home equity loan,student loan
0,CUS_0x1000,30625.939453,2706.161621,6,5,27.0,2,57,26,1.630000,...,0.015868,0,0,0,0,0,1,0,1,0
1,CUS_0x1009,52312.679688,4250.390137,6,5,17.0,4,5,18,9.730000,...,0.025496,0,0,1,0,1,1,0,1,0
2,CUS_0x1011,58918.468750,5208.872559,3,3,17.0,3,27,13,14.420000,...,0.023697,1,0,0,0,0,1,0,0,1
3,CUS_0x1013,98620.976562,7962.415039,3,3,6.0,3,12,9,1.330000,...,0.028637,1,1,0,0,0,0,0,0,1
4,CUS_0x1018,61194.808594,5014.567383,7,7,23.0,8,19,22,28.629999,...,0.044943,0,1,1,0,1,1,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9768,CUS_0xff3,17032.785156,1176.398804,0,6,2.0,3,13,7,6.860000,...,0.028307,0,1,0,1,0,0,1,0,0
9769,CUS_0xff4,25546.259766,2415.854980,8,7,14.0,5,15,13,7.830000,...,0.041943,0,0,0,0,1,1,1,0,1
9770,CUS_0xff6,117639.921875,9727.327148,5,6,1.0,2,-3,7,10.400000,...,0.013019,0,0,0,0,0,0,1,1,0
9771,CUS_0xffc,60877.171875,5218.097656,6,8,27.0,8,46,14,8.820000,...,0.052281,0,0,1,0,1,1,0,1,1


In [189]:
# Process Payment_Behaviour
df = df.withColumn("Payment_Behaviour_Types", split(col("Payment_Behaviour"), ", and |, "))
unique_payment_behaviours = df.select(explode(col("Payment_Behaviour_Types")).alias("Payment_Behaviour_Type")).distinct().collect()
unique_payment_behaviours = [row["Payment_Behaviour_Type"] for row in unique_payment_behaviours]
for behaviour in unique_payment_behaviours:
    df = df.withColumn("Payment_" + behaviour, array_contains(col("Payment_Behaviour_Types"), behaviour).cast("int"))
df = df.drop("Payment_Behaviour_Types")
df.toPandas()

,Customer_ID,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,Num_of_Loan,Delay_from_due_date,Num_of_Delayed_Payment,Changed_Credit_Limit,...,credit-builder loan,auto loan,home equity loan,student loan,Payment_high_spent_large_value_payments,Payment_low_spent_medium_value_payments,Payment_high_spent_medium_value_payments,Payment_high_spent_small_value_payments,Payment_low_spent_small_value_payments,Payment_low_spent_large_value_payments
0,CUS_0x1000,30625.939453,2706.161621,6,5,27.0,2,57,26,1.630000,...,1,0,1,0,0,0,1,0,0,0
1,CUS_0x1009,52312.679688,4250.390137,6,5,17.0,4,5,18,9.730000,...,1,0,1,0,0,0,1,0,0,0
2,CUS_0x1011,58918.468750,5208.872559,3,3,17.0,3,27,13,14.420000,...,1,0,0,1,0,1,0,0,0,0
3,CUS_0x1013,98620.976562,7962.415039,3,3,6.0,3,12,9,1.330000,...,0,0,0,1,0,0,1,0,0,0
4,CUS_0x1018,61194.808594,5014.567383,7,7,23.0,8,19,22,28.629999,...,1,0,1,1,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9768,CUS_0xff3,17032.785156,1176.398804,0,6,2.0,3,13,7,6.860000,...,0,1,0,0,0,0,0,0,1,0
9769,CUS_0xff4,25546.259766,2415.854980,8,7,14.0,5,15,13,7.830000,...,1,1,0,1,0,1,0,0,0,0
9770,CUS_0xff6,117639.921875,9727.327148,5,6,1.0,2,-3,7,10.400000,...,0,1,1,0,0,1,0,0,0,0
9771,CUS_0xffc,60877.171875,5218.097656,6,8,27.0,8,46,14,8.820000,...,1,0,1,1,1,0,0,0,0,0


In [190]:
unique_payment_behaviours

['high_spent_large_value_payments',
 'low_spent_medium_value_payments',
 'high_spent_medium_value_payments',
 'high_spent_small_value_payments',
 'low_spent_small_value_payments',
 'low_spent_large_value_payments']

In [191]:
# Process Credit_Mix
df = df.withColumn("Credit_Mix_Types", split(col("Credit_Mix"), ", and |, "))
unique_credit_mixes = df.select(explode(col("Credit_Mix_Types")).alias("Credit_Mix_Type")).distinct().collect()
unique_credit_mixes = [row["Credit_Mix_Type"] for row in unique_credit_mixes]
for mix in unique_credit_mixes:
    df = df.withColumn("Credit_Mix_" + mix, array_contains(col("Credit_Mix_Types"), mix).cast("int"))
df = df.drop("Credit_Mix_Types")

In [192]:
df.toPandas()

,Customer_ID,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,Num_of_Loan,Delay_from_due_date,Num_of_Delayed_Payment,Changed_Credit_Limit,...,student loan,Payment_high_spent_large_value_payments,Payment_low_spent_medium_value_payments,Payment_high_spent_medium_value_payments,Payment_high_spent_small_value_payments,Payment_low_spent_small_value_payments,Payment_low_spent_large_value_payments,Credit_Mix_bad,Credit_Mix_standard,Credit_Mix_good
0,CUS_0x1000,30625.939453,2706.161621,6,5,27.0,2,57,26,1.630000,...,0,0,0,1,0,0,0,1.0,0.0,0.0
1,CUS_0x1009,52312.679688,4250.390137,6,5,17.0,4,5,18,9.730000,...,0,0,0,1,0,0,0,NaN,NaN,NaN
2,CUS_0x1011,58918.468750,5208.872559,3,3,17.0,3,27,13,14.420000,...,1,0,1,0,0,0,0,0.0,1.0,0.0
3,CUS_0x1013,98620.976562,7962.415039,3,3,6.0,3,12,9,1.330000,...,1,0,0,1,0,0,0,0.0,0.0,1.0
4,CUS_0x1018,61194.808594,5014.567383,7,7,23.0,8,19,22,28.629999,...,1,0,0,1,0,0,0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9768,CUS_0xff3,17032.785156,1176.398804,0,6,2.0,3,13,7,6.860000,...,0,0,0,0,0,1,0,0.0,0.0,1.0
9769,CUS_0xff4,25546.259766,2415.854980,8,7,14.0,5,15,13,7.830000,...,1,0,1,0,0,0,0,0.0,1.0,0.0
9770,CUS_0xff6,117639.921875,9727.327148,5,6,1.0,2,-3,7,10.400000,...,0,0,1,0,0,0,0,0.0,0.0,1.0
9771,CUS_0xffc,60877.171875,5218.097656,6,8,27.0,8,46,14,8.820000,...,1,1,0,0,0,0,0,NaN,NaN,NaN


In [193]:
unique_credit_mixes

['bad', 'standard', 'good']

In [194]:
# Process Payment_of_Min_Amount
df = df.withColumn("Payment_of_Min_Amount_Types", split(col("Payment_of_Min_Amount"), ", and |, "))
unique_payment_of_min_amounts = df.select(explode(col("Payment_of_Min_Amount_Types")).alias("Payment_of_Min_Amount_Type")).distinct().collect()
unique_payment_of_min_amounts = [row["Payment_of_Min_Amount_Type"] for row in unique_payment_of_min_amounts]
for payment in unique_payment_of_min_amounts:
    df = df.withColumn("Payment_of_Min_" + payment, array_contains(col("Payment_of_Min_Amount_Types"), payment).cast("int"))
df = df.drop("Payment_of_Min_Amount_Types")
df.toPandas()

,Customer_ID,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,Num_of_Loan,Delay_from_due_date,Num_of_Delayed_Payment,Changed_Credit_Limit,...,Payment_high_spent_medium_value_payments,Payment_high_spent_small_value_payments,Payment_low_spent_small_value_payments,Payment_low_spent_large_value_payments,Credit_Mix_bad,Credit_Mix_standard,Credit_Mix_good,Payment_of_Min_nm,Payment_of_Min_no,Payment_of_Min_yes
0,CUS_0x1000,30625.939453,2706.161621,6,5,27.0,2,57,26,1.630000,...,1,0,0,0,1.0,0.0,0.0,0,0,1
1,CUS_0x1009,52312.679688,4250.390137,6,5,17.0,4,5,18,9.730000,...,1,0,0,0,NaN,NaN,NaN,0,0,1
2,CUS_0x1011,58918.468750,5208.872559,3,3,17.0,3,27,13,14.420000,...,0,0,0,0,0.0,1.0,0.0,0,0,1
3,CUS_0x1013,98620.976562,7962.415039,3,3,6.0,3,12,9,1.330000,...,1,0,0,0,0.0,0.0,1.0,0,1,0
4,CUS_0x1018,61194.808594,5014.567383,7,7,23.0,8,19,22,28.629999,...,1,0,0,0,1.0,0.0,0.0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9768,CUS_0xff3,17032.785156,1176.398804,0,6,2.0,3,13,7,6.860000,...,0,0,1,0,0.0,0.0,1.0,0,1,0
9769,CUS_0xff4,25546.259766,2415.854980,8,7,14.0,5,15,13,7.830000,...,0,0,0,0,0.0,1.0,0.0,0,0,1
9770,CUS_0xff6,117639.921875,9727.327148,5,6,1.0,2,-3,7,10.400000,...,0,0,0,0,0.0,0.0,1.0,0,1,0
9771,CUS_0xffc,60877.171875,5218.097656,6,8,27.0,8,46,14,8.820000,...,0,0,0,0,NaN,NaN,NaN,0,0,1


In [195]:
unique_payment_of_min_amounts

['nm', 'no', 'yes']

In [197]:
df = df.na.drop()
df.toPandas()

,Customer_ID,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,Num_of_Loan,Delay_from_due_date,Num_of_Delayed_Payment,Changed_Credit_Limit,...,Payment_high_spent_medium_value_payments,Payment_high_spent_small_value_payments,Payment_low_spent_small_value_payments,Payment_low_spent_large_value_payments,Credit_Mix_bad,Credit_Mix_standard,Credit_Mix_good,Payment_of_Min_nm,Payment_of_Min_no,Payment_of_Min_yes
0,CUS_0x1000,30625.939453,2706.161621,6,5,27.0,2,57,26,1.630000,...,1,0,0,0,1,0,0,0,0,1
1,CUS_0x1011,58918.468750,5208.872559,3,3,17.0,3,27,13,14.420000,...,0,0,0,0,0,1,0,0,0,1
2,CUS_0x1013,98620.976562,7962.415039,3,3,6.0,3,12,9,1.330000,...,1,0,0,0,0,0,1,0,1,0
3,CUS_0x1018,61194.808594,5014.567383,7,7,23.0,8,19,22,28.629999,...,1,0,0,0,1,0,0,0,0,1
4,CUS_0x1026,170614.281250,14463.856445,2,6,9.0,2,0,5,0.730000,...,1,0,0,0,0,0,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7191,CUS_0xfea,44264.410156,3755.700928,3,3,6.0,2,11,2,9.660000,...,0,0,1,0,0,0,1,0,1,0
7192,CUS_0xff3,17032.785156,1176.398804,0,6,2.0,3,13,7,6.860000,...,0,0,1,0,0,0,1,0,1,0
7193,CUS_0xff4,25546.259766,2415.854980,8,7,14.0,5,15,13,7.830000,...,0,0,0,0,0,1,0,0,0,1
7194,CUS_0xff6,117639.921875,9727.327148,5,6,1.0,2,-3,7,10.400000,...,0,0,0,0,0,0,1,0,1,0
